## EDA를 통한 데이터 축소
---
#### 추가 변수입력 상태에서 데이터를 정제하고 쓸만한 데이터만을 추출하기 위해 EDA 및 변수 선택
1. 데이터 및 패키지 로드
  - drop_data
  ---
2. 정량, 정성 데이터 구분
  - 보증금, 월세금 등을 수치형(정수)으로 변환
  - 범주형 데이터의 unique 확인
  ---
3. EDA 진행
  - 수치형과 범주형을 나눠서함
  - 수치형의 경우 이상치 선정
  - 범주형의 경우 데이터 편향 고려

4. 주택 유형별로 이븐하게 선정

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#### 패키지로드

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/predict_price/MergedData/drop_data.csv")

In [ ]:
df.head()

,Unnamed: 0,단지명,전월세구분,전용면적(㎡),보증금(만원),월세금(만원),층,건축년도,계약구분,갱신요구권 사용,...,동,계약날짜,계약연도,계약월,계약요일,IMF_경과년수,금융위기_경과년수,계약개월수,단지별 평균 보증금,단지별 평균 월세금
0,0,청호 서울숲 뜨레피움 오피스텔,전세,19.03,18000.0,0.0,3,2018.0,-,0,...,성수동1가,2019-04-30,2019,4,1,21.424658,10.627397,24,18545.588235,0.00000
1,1,더클래스,전세,43.17,23000.0,0.0,4,2019.0,-,0,...,불광동,2019-04-30,2019,4,1,21.424658,10.627397,24,22627.794118,0.00000
2,2,화양타워,전세,37.34,11000.0,0.0,4,1999.0,-,0,...,화양동,2019-04-30,2019,4,1,21.424658,10.627397,24,18908.888889,0.00000
3,3,효성네오인텔리안,월세,31.71,1000.0,50.0,4,2005.0,-,0,...,번동,2019-04-30,2019,4,1,21.424658,10.627397,24,655.172414,57.37931
4,4,한강현대하이엘,전세,40.28,21000.0,0.0,15,2005.0,-,0,...,한강로2가,2019-04-30,2019,4,1,21.424658,10.627397,24,21936.083333,0.00000


In [ ]:
col = df.columns

In [ ]:
col


Index(['시군구', '단지명', '전월세구분', '전용면적(㎡)', '계약년월', '계약일', '보증금(만원)', '월세금(만원)',
       '층', '건축년도', '도로명', '계약기간', '계약구분', '갱신요구권 사용', '종전계약 보증금(만원)',
       '종전계약 월세(만원)', '주택유형', 'index', 'rate', 'pp_rate', '시군구_수정', '전체주소'],
      dtype='object')

In [ ]:
df['주택유형'].describe()
df['주택유형'].unique()

array(['오피스텔', '아파트', '연립다세대'], dtype=object)

In [ ]:
df['주택유형'].value_counts()

,count
주택유형,
아파트,1685414
오피스텔,157784
연립다세대,49689


In [ ]:
df['계약구분'].value_counts()

,count
계약구분,
-,881719
신규,634639
갱신,376529


In [ ]:
# 종전 계약이 NaN인 경우, 계약구분이 '신규'인지 확인
inconsistent_rows = df[(df["종전계약 보증금(만원)"].isna()) & (df["계약구분"] == "신규")]

# 불일치한 데이터 출력
print(len(df), len(inconsistent_rows))


1892887 634639


In [ ]:
df = df.drop(columns=["시군구_수정"])

In [ ]:
df.dtypes

,0
시군구,object
단지명,object
전월세구분,object
전용면적(㎡),float64
계약년월,object
계약일,int64
보증금(만원),object
월세금(만원),int64
층,int64
건축년도,float64


In [ ]:
df['보증금(만원)'] = df['보증금(만원)'].str.replace(',', '').astype(int)

In [ ]:
df['건축년도'] = df['건축년도'].astype(int)

In [ ]:
df['계약일'] = df['계약일'].astype(str)
for i in range(len(df)):
  if len(df['계약일'][i]) == 1:
    df['계약일'][i] = "0" + df['계약일'][i]

In [ ]:
df["계약날짜"] = pd.to_datetime(df["계약년월"].astype(str) + "-" + df["계약일"].astype(str), format="%Y-%m-%d")
df['계약날짜']

,계약날짜
0,2019-04-30
1,2019-04-30
2,2019-04-30
3,2019-04-30
4,2019-04-30
...,...
1892882,2024-05-01
1892883,2024-05-01
1892884,2024-05-01
1892885,2024-05-01


In [ ]:
df = df.drop(columns=["계약년월", "계약일"])


In [ ]:
df.isnull().sum()

,0
시군구,0
단지명,49689
전월세구분,2
전용면적(㎡),0
보증금(만원),0
월세금(만원),0
층,0
건축년도,0
도로명,0
계약기간,0


In [ ]:
df.drop()

In [ ]:
last_df = df.copy()

In [ ]:
df_filtered = last_df.dropna(subset=["종전계약 보증금(만원)"])


In [ ]:
df_filtered = df_filtered.dropna(subset=["단지명", "전월세구분"])

In [ ]:
df_filtered.reset_index(drop=True, inplace=True)

In [ ]:
df_filtered.isnull().sum()

,0
시군구,0
단지명,0
전월세구분,0
전용면적(㎡),0
보증금(만원),0
월세금(만원),0
층,0
건축년도,0
도로명,0
계약기간,0


#### 최종 정리 (전처리 순서)
- 1️⃣ 범주형 데이터 변환

  - 원-핫 인코딩 or 레이블 인코딩
- 2️⃣ 숫자형 데이터 처리
  - 결측치 채우기 (fillna())
  - 이상치 제거 (IQR)
- 3️⃣ 날짜형 데이터 변환
  - 연도/월/일 분리
  - 최근성과 차이 계산
- 4️⃣ 주소 데이터 정리
  - 도로명만 추출
  - 시군구 레이블 인코딩
- 5️⃣ 정규화/표준화
  - MinMaxScaler() or StandardScaler()

#### 수치형 데이터 처리

In [ ]:
def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

df_filtered = remove_outliers(df_filtered, "보증금(만원)")
df_filtered = remove_outliers(df_filtered, "월세금(만원)")
df_filtered = remove_outliers(df_filtered, "전용면적(㎡)")

In [ ]:
# df["보증금(만원)"].fillna(0, inplace=True)
# df["월세금(만원)"].fillna(0, inplace=True)
# df["층"].fillna(df["층"].median(), inplace=True)
# df["건축년도"].fillna(df["건축년도"].median(), inplace=True)
# df["rate"].fillna(df["rate"].mean(), inplace=True)

#### 범주형 데이터 처

In [ ]:
# One-Hot Encoding (추천)
df_filtered = pd.get_dummies(df_filtered, columns=["전월세구분", "계약구분", "갱신요구권 사용", "주택유형"])
from sklearn.preprocessing import LabelEncoder

# # Label Encoding 예시 (순서 없는 변수는 원-핫 인코딩이 더 나음)
# categorical_cols = ["전월세구분", "계약구분", "갱신요구권 사용", "주택유형"]
# for col in categorical_cols:
#     le = LabelEncoder()
#     df[col] = le.fit_transform(df[col])


#### 날짜 데이터


In [ ]:
df_filtered["계약연"] = df_filtered["계약날짜"].dt.year
df_filtered["계약월"] = df_filtered["계약날짜"].dt.month

#### 정규화

In [ ]:

from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Min-Max 정규화 (0~1)
scaler = MinMaxScaler()
df_filtered[["보증금(만원)", "월세금(만원)", "전용면적(㎡)", "층"]] = scaler.fit_transform(df_filtered[["보증금(만원)", "월세금(만원)", "전용면적(㎡)", "층"]])

# StandardScaler (표준화)
scaler = StandardScaler()
df_filtered[["index", "rate", "pp_rate"]] = scaler.fit_transform(df_filtered[["index", "rate", "pp_rate"]])


#### Save data

In [ ]:
df_filtered.to_csv("/content/drive/MyDrive/Colab Notebooks/predict_price/MergedData/final_data.csv", index=False)